# Pancreas Demo: Path-Local / Path-Global / Gate Inference

This tutorial runs the three released Nuclass backbones on the same pancreas patch set.
We reuse the sanitized inference modules under `Nuclass.test` so the code here stays identical to the paper release.

### Data & Checkpoints
- **Demo data**: `Nuclass/demo/pancreas` (contains `annotations.json`, `patches_224x224.h5`, `patches_1024x1024.h5`).
- **Checkpoints**: `Nuclass/checkpoints/Path_local.ckpt`, `Path_global.ckpt`, and `Gate.ckpt`.

Outputs from each backbone are written to `Nuclass/demo/pancreas/inference_outputs/...`.

In [ ]:
import sys
from pathlib import Path

notebook_dir = Path.cwd().resolve()
repo_root = None
for candidate in [notebook_dir, *notebook_dir.parents]:
    if (candidate / 'Nuclass' / '__init__.py').exists():
        repo_root = candidate
        break

if repo_root is None:
    raise RuntimeError('Unable to find the Nuclass package root; please update the search logic above.')
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))
print(f'Python path synced with repo: {repo_root}')


Python path synced with repo: /data/Data_for_Nuclass_training


In [18]:
from pathlib import Path
import json
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import torch
from IPython.display import display

try:
    REPO_ROOT = Path(repo_root)
except NameError as exc:
    raise RuntimeError('Run the path-setup cell above before importing packages.') from exc

BASE_DIR = REPO_ROOT / 'Nuclass'
DATA_ROOT = BASE_DIR / 'demo' / 'pancreas'
CKPT_PATH_LOCAL = BASE_DIR / 'checkpoints' / 'Path_local.ckpt'
CKPT_PATH_GLOBAL = BASE_DIR / 'checkpoints' / 'Path_global.ckpt'
CKPT_PATH_GATE = BASE_DIR / 'checkpoints' / 'Gate.ckpt'
OUTPUT_ROOT = DATA_ROOT / 'inference_outputs'
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if DEVICE == 'cuda':
    torch.cuda.set_device(0)
    print('Using GPU:', torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print('Using CPU for inference')


Using GPU: NVIDIA H200 NVL


In [19]:
import importlib
A_module = importlib.reload(importlib.import_module('Nuclass.test.local_infrence'))
B_module = importlib.reload(importlib.import_module('Nuclass.test.global_inference'))
Gate_module = importlib.reload(importlib.import_module('Nuclass.test.gate_inference'))

### Helper functions
The helpers below wrap the CLI-friendly inference utilities so we can call them directly inside the notebook.

In [20]:
def summarize_metrics(metrics: dict, label: str) -> dict:
    summary = {'model': label, 'samples': metrics.get('n_samples')}
    if 'acc_mix' in metrics:
        summary['accuracy'] = metrics.get('acc_mix')
        summary['macro_f1'] = metrics.get('f1_mix')
        summary['acc_branch_local'] = metrics.get('acc_local')
        summary['acc_branch_global'] = metrics.get('acc_global')
        if metrics.get('acc_safe') is not None:
            summary['acc_safe'] = metrics.get('acc_safe')
            summary['f1_safe'] = metrics.get('f1_safe')
    else:
        summary['accuracy'] = metrics.get('overall_acc')
        summary['macro_f1'] = metrics.get('macro_f1')
    return summary


def run_path_a(device: str = DEVICE) -> Path:
    out_dir = OUTPUT_ROOT / 'path_local'
    model, train_names, _ = A_module.load_model_from_ckpt(str(CKPT_PATH_LOCAL), device=device)
    A_module.infer_one_holdout(
        model,
        train_names,
        holdout_name='pancreas_demo',
        root=str(DATA_ROOT),
        mode='xenium_exact',
        tissue='pancreas',
        out_dir=str(out_dir),
        device=device,
    )
    return out_dir


def run_path_b(device: str = DEVICE, ctx_size: int = 512, batch: int = 256) -> Path:
    out_dir = OUTPUT_ROOT / 'path_global'
    raw = torch.load(str(CKPT_PATH_GLOBAL), map_location='cpu')
    hp = raw.get('hyper_parameters', {})
    class_names = hp.get('class_names')
    if not class_names:
        raise RuntimeError('Path-global ckpt missing hyper_parameters.class_names')
    num_tissues = int(hp.get('num_tissues', 8))
    dino_name = hp.get('config', {}).get('dino_model', 'facebook/dinov3-vitl16-pretrain-lvd1689m')
    model = B_module.PathBInfer(num_classes=len(class_names), num_tissues=num_tissues,
                                dino_model_name=dino_name, ctx_mb=32)
    model.load_from_lightning_ckpt(str(CKPT_PATH_GLOBAL), map_location='cpu')
    model.to(device).eval()
    B_module.infer_holdout(
        model,
        class_names,
        name='pancreas_demo',
        root=str(DATA_ROOT),
        mode='xenium_exact',
        tissue='pancreas',
        out_dir=str(out_dir),
        ctx_size=ctx_size,
        batch=batch,
    )
    return out_dir


def run_gate_mix(device: str = DEVICE, ctx_size: int = 384, batch: int = 128) -> Path:
    out_dir = OUTPUT_ROOT / 'gate_mix'
    raw = torch.load(str(CKPT_PATH_GATE), map_location='cpu')
    hp = raw.get('hyper_parameters', {})
    cfg = hp.get('config', {})
    class_names = hp.get('class_names')
    if not class_names:
        ckptA = hp.get('ckptA') or str(CKPT_PATH_LOCAL)
        aux = torch.load(ckptA, map_location='cpu')
        class_names = aux.get('hyper_parameters', {}).get('class_names')
    if not class_names:
        raise RuntimeError('Unable to recover training class_names for Gate ckpt.')
    num_tissues = int(hp.get('num_tissues', 8))
    dino_name = cfg.get('dino_model', 'facebook/dinov3-vitl16-pretrain-lvd1689m')
    gdimA = int(cfg.get('gate_proj_dim_A', 128))
    gdimB = int(cfg.get('gate_proj_dim_B', 128))
    ctx_mb = int(cfg.get('ctx_microbatch', 16))

    model = Gate_module.FuseABGateInfer(
        num_classes=len(class_names),
        num_tissues=num_tissues,
        dino_model=dino_name,
        gate_dim_A=gdimA,
        gate_dim_B=gdimB,
        ctx_mb=ctx_mb,
    )
    model.load_from_fuse_ckpt(str(CKPT_PATH_GATE), map_location='cpu')
    model.to(device).eval()

    Gate_module.infer_holdout(
        model,
        class_names,
        name='pancreas_demo',
        root=str(DATA_ROOT),
        mode='xenium_exact',
        tissue='pancreas',
        out_dir=str(out_dir),
        ctx_size=ctx_size,
        batch=batch,
    )
    return out_dir


def load_metrics(out_dir: Path, label: str) -> dict:
    metrics_path = Path(out_dir) / 'metrics.json'
    with metrics_path.open('r') as f:
        metrics = json.load(f)
    return summarize_metrics(metrics, label)


def load_per_class_f1_table(out_dir: Path, label: str) -> pd.DataFrame:
    out_dir = Path(out_dir)
    candidates = [
        'aligned_per_class_f1.csv',
        'aligned_mix_per_class_f1.csv',
        'per_class_f1.csv',
    ]
    csv_path = None
    for name in candidates:
        cand = out_dir / name
        if cand.exists():
            csv_path = cand
            break
    if csv_path is not None:
        df = pd.read_csv(csv_path)
        class_col = next((c for c in df.columns if c.lower().startswith('class')), df.columns[0])
        f1_col = next((c for c in df.columns if 'f1' in c.lower()), df.columns[-1])
        per_class = pd.DataFrame({'class': df[class_col], 'f1': df[f1_col]})
    else:
        y_true_path = out_dir / 'y_true.npy'
        y_pred_path = out_dir / 'y_pred.npy'
        labels_path = out_dir / 'labels.npy'
        preds_path = out_dir / 'preds.npy'
        if y_true_path.exists():
            y_true = np.load(y_true_path)
            y_pred = np.load(y_pred_path)
        elif labels_path.exists():
            y_true = np.load(labels_path)
            y_pred = np.load(preds_path)
        else:
            raise FileNotFoundError(f'Could not find label files in {out_dir}')
        names_path = out_dir / 'class_names_used.json'
        if not names_path.exists():
            raise FileNotFoundError(names_path)
        with names_path.open('r') as f:
            class_names = json.load(f)
        scores = f1_score(y_true, y_pred, labels=list(range(len(class_names))), average=None, zero_division=0)
        per_class = pd.DataFrame({'class': class_names, 'f1': scores})
    per_class.insert(0, 'model', label)
    return per_class


### Run inference
Execute the cell below to run Path-Local, Path-Global, and the fusion gate on the pancreas dataset.
The DINO-based models may take a few minutes.

In [21]:
from tqdm.auto import tqdm

run_specs = [
    ("Path-Local (UNI + FiLM)", run_path_a, (), {}, "path_a_dir"),
    ("Path-Global (UNI + DINO)", run_path_b, (), {}, "path_b_dir"),
    ("GateMix (Fusion)", run_gate_mix, (), {}, "gate_dir"),
]

outputs = {}
with tqdm(total=len(run_specs), desc="Running pancreas demo", unit="model") as overall:
    for label, fn, args, kwargs, key in run_specs:
        with tqdm(total=1, desc=label, leave=False, bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt}") as step_bar:
            outputs[key] = fn(*args, **kwargs)
            step_bar.update(1)
        tqdm.write(f"{label} complete -> {outputs[key]}")
        overall.update(1)

path_a_dir = outputs["path_a_dir"]
path_b_dir = outputs["path_b_dir"]
gate_dir = outputs["gate_dir"]

print('Outputs saved to:')
print('  Path-Local :', path_a_dir)
print('  Path-Global :', path_b_dir)
print('  GateMix:', gate_dir)



Running pancreas demo:   0%|          | 0/3 [00:00<?, ?model/s]

Path-Local (UNI + FiLM):   0%|          | 0/1

[ckpt] unexpected(2): ['class_weights', 'criterion.weight'] ...
  - Reading annotations from /data/Data_for_Nuclass_training/Nuclass/demo/pancreas/annotations.json
  - Dropped (not in training label space) (10)
               Metaplastic Cells : 5525
                     Tumor Cells : 4127
                     Endocrine 2 : 3887
                     Endocrine 1 : 2700
               CFTR- Tumor Cells : 2368
                      Mast Cells : 426
                  CXCL9/10 Cells : 383
                         B Cells : 206
             Smooth Muscle Cells : 153
     Lymphatic Endothelial Cells : 93
  - Samples=14213 | classes before mapping=16 | after mapping=6
  - Label counts before mapping (16)
                          Acinar : 5615
               Metaplastic Cells : 5525
                     Tumor Cells : 4127
                     Endocrine 2 : 3887
                      Macrophage : 2867
                     Endocrine 1 : 2700
                      Fibroblast : 2573
              

[pancreas_demo] infer:   0%|          | 0/14 [00:00<?, ?it/s]

  -> saved: /data/Data_for_Nuclass_training/Nuclass/demo/pancreas/inference_outputs/path_local | acc=0.7291 f1m=0.6974
Path-Local (UNI + FiLM) complete -> /data/Data_for_Nuclass_training/Nuclass/demo/pancreas/inference_outputs/path_local


...

[pancreas_demo] acc=0.7677  f1_macro=0.7575
Path-Global (UNI + DINO) complete -> /data/Data_for_Nuclass_training/Nuclass/demo/pancreas/inference_outputs/path_global


GateMix (Fusion):   0%|          | 0/1

[ckpt] modelA          | loaded= 354  missing= 0  unexpected= 0
[ckpt] modelB          | loaded= 770  missing= 0  unexpected= 0
[ckpt] gate_proj_A     | loaded=   4  missing= 0  unexpected= 0
[ckpt] gate_proj_B     | loaded=   4  missing= 0  unexpected= 0
[ckpt] gate_mlp        | loaded=   6  missing= 0  unexpected= 0
[ckpt] load done.
  - Reading annotations from /data/Data_for_Nuclass_training/Nuclass/demo/pancreas/annotations.json
  - Dropped (not present in training label space):
      Metaplastic Cells            : 5525
      Tumor Cells                  : 4127
      Endocrine 2                  : 3887
      Endocrine 1                  : 2700
      CFTR- Tumor Cells            : 2368
      Mast Cells                   : 426
      CXCL9/10 Cells               : 383
      B Cells                      : 206
      Smooth Muscle Cells          : 153
      Lymphatic Endothelial Cells  : 93
  - Column alignment (used -> train_col):
      Acinar                       -> train_col 0
     

[pancreas_demo] infer:   0%|                                                | 0/112 [00:00<?, ?it/s]

[pancreas_demo] A(acc=0.7291,f1=0.6974) | B(acc=0.7340,f1=0.7108) | Mix(acc=0.7367,f1=0.7116)
GateMix (Fusion) complete -> /data/Data_for_Nuclass_training/Nuclass/demo/pancreas/inference_outputs/gate_mix
Outputs saved to:
  Path-Local : /data/Data_for_Nuclass_training/Nuclass/demo/pancreas/inference_outputs/path_local
  Path-Global : /data/Data_for_Nuclass_training/Nuclass/demo/pancreas/inference_outputs/path_global
  GateMix: /data/Data_for_Nuclass_training/Nuclass/demo/pancreas/inference_outputs/gate_mix


### Compare metrics

In [26]:
from tqdm.auto import tqdm

metric_jobs = [
    ('Path-Local (UNI + FiLM)', path_a_dir),
    ('Path-Global (UNI + DINO)', path_b_dir),
    ('GateMix (Fusion)', gate_dir),
]

summary = []
for label, directory in tqdm(metric_jobs, desc='Collecting metrics', unit='model'):
    summary.append(load_metrics(directory, label))

results_df = pd.DataFrame(summary)
overall = results_df[['model','accuracy','macro_f1']].copy()
print('Overall accuracy / macro-F1')
print(overall.to_string(index=False))




Overall accuracy / macro-F1
                   model  accuracy  macro_f1
 Path-Local (UNI + FiLM)  0.729051  0.697401
Path-Global (UNI + DINO)  0.767730  0.757468
        GateMix (Fusion)  0.736720  0.711637
